In [2]:
from openai import OpenAI

# openai client will automatically read the OPENAI_API_KEY environment
# variable, but you can also set it manually like this:
# openai =
# OpenAI(api_key="<api key>")
openai = OpenAI()

In [3]:
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a brief paragraph about the history of Tennessee.",
        },
    ],
    temperature=0.6,
    max_tokens=150,
)

In [4]:
response.choices[0].message.content

"Tennessee's history is rich and varied, beginning with its indigenous peoples, including the Cherokee, Chickasaw, and Creek tribes. European exploration began in the mid-16th century with Spanish expeditions, but it was not until the late 18th century that settlers from the eastern colonies began to establish permanent settlements. Tennessee was originally part of North Carolina and later the Southwest Territory before becoming the 16th state of the United States on June 1, 1796. During the Civil War, Tennessee was a significant battleground, being the last state to join the Confederacy and the first to be readmitted to the Union. The state played a crucial role in the civil rights movement in the 20th century, highlighted by events"

In [9]:
print(response)

ChatCompletion(id='chatcmpl-ALB9QziYu4Zqmm4YW61JS6ysxkbEi', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="Tennessee's history is rich and varied, beginning with its indigenous peoples, including the Cherokee, Chickasaw, and Creek tribes. European exploration began in the mid-16th century with Spanish expeditions, but it was not until the late 18th century that settlers from the eastern colonies began to establish permanent settlements. Tennessee was originally part of North Carolina and later the Southwest Territory before becoming the 16th state of the United States on June 1, 1796. During the Civil War, Tennessee was a significant battleground, being the last state to join the Confederacy and the first to be readmitted to the Union. The state played a crucial role in the civil rights movement in the 20th century, highlighted by events", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=

In [14]:
%pip install langchain langchain-community langchain-openai

  Using cached regex-2024.9.11-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 6.6 MB/s eta 0:00:00
Using cached regex-2024.9.11-cp311-cp311-macosx_11_0_arm64.whl (284 kB)
Note: you may need to restart the kernel to use updated packages.


In [35]:
from langchain import PromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# Database stub
database = {
    "invoice_001": {
        "supplier": "ABC Corp",
        "amount": 1500,
        "date": "2023-10-01",
        "status": "Paid",
    },
    "invoice_002": {
        "supplier": "XYZ Ltd",
        "amount": 2500,
        "date": "2023-10-05",
        "status": "Pending",
    },
}


# Function to fetch invoice information from the database
@tool
def fetch_invoice_info(invoice_id):
    """Fetch invoice information from the database."""
    return database.get(invoice_id, "Invoice not found")


# Define our tools list
tools = [fetch_invoice_info]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["invoice_id"],
    template="What's the status of invoice id {invoice_id}?",
)

# Initialize the model
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

# Call model w/ prompt and pass specific invoice_id
response = llm_with_tools.invoke(prompt_template.format(invoice_id="invoice_001"))

In [36]:
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_nNrOSkfwHzUTCJLGXl5uWUZQ', 'function': {'arguments': '{"invoice_id":"invoice_001"}', 'name': 'fetch_invoice_info'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 56, 'total_tokens': 74, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a7d06e42a7', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-28342eff-867c-4a0f-9d80-a91fb67349a0-0' tool_calls=[{'name': 'fetch_invoice_info', 'args': {'invoice_id': 'invoice_001'}, 'id': 'call_nNrOSkfwHzUTCJLGXl5uWUZQ', 'type': 'tool_call'}] usage_metadata={'input_tokens': 56, 'output_tokens': 18, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


In [37]:
messages = [response]

In [38]:
for tool_call in response.tool_calls:
    selected_tool = {"fetch_invoice_info": fetch_invoice_info}[
        tool_call["name"].lower()
    ]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

In [39]:
print(messages)
messages.append(llm_with_tools.invoke(messages))

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nNrOSkfwHzUTCJLGXl5uWUZQ', 'function': {'arguments': '{"invoice_id":"invoice_001"}', 'name': 'fetch_invoice_info'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 56, 'total_tokens': 74, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a7d06e42a7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-28342eff-867c-4a0f-9d80-a91fb67349a0-0', tool_calls=[{'name': 'fetch_invoice_info', 'args': {'invoice_id': 'invoice_001'}, 'id': 'call_nNrOSkfwHzUTCJLGXl5uWUZQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 18, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), ToolMessage(content='{"supplier": "ABC Corp", "

In [42]:
print(messages)

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nNrOSkfwHzUTCJLGXl5uWUZQ', 'function': {'arguments': '{"invoice_id":"invoice_001"}', 'name': 'fetch_invoice_info'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 56, 'total_tokens': 74, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a7d06e42a7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-28342eff-867c-4a0f-9d80-a91fb67349a0-0', tool_calls=[{'name': 'fetch_invoice_info', 'args': {'invoice_id': 'invoice_001'}, 'id': 'call_nNrOSkfwHzUTCJLGXl5uWUZQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 18, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), ToolMessage(content='{"supplier": "ABC Corp", "